# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a gym to open based on impacts of COVID-19. Specifically, this report will be targeted to fitness fanatics interested in going to a **gym or fitness centre** in **Melbourne**, Victoria.

Since there are lots of gyms in Melbourne we will try to detect **locations that were impacted by high cases of COVID-19**. We are also particularly interested in **areas with high case counts**. We would also prefer locations **as close to city center as possible**, assuming that most people are located near the CBD.

I will use my Data Sceince learnings to create map of the gyms in high impacted areas.


## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our dicussion are:
* number of total cases of COVID-19 in Victorian muncipalities  (any type of postive test recorded as of June 7th 2020)
* number of active cases of COVID-19 in Victorian muncipalities (as of June 7th 2020)
* gym and fitness centres locations in municipalities across Victoria


Following data sources will be needed to extract/generate the required information:
*  **Department of Health and Human Services COVID-19 Daily Breifings report** from the ** Department of Health and Human Sercvices** website.
* number of gyms and fitness centres and their type and location in every municipality will be obtained using **Foursquare API**
* coordinate of Melbourne CBD center will be obtained using **Foursquare API geocoding** of locations

## Methodology <a name="methodology"></a>

1. Collect data source/inputs/inspect the data/set up environments
2. Explore and understand the data
3. Cleanse and enhance the data
4. Model the data

**1. Collect data source/inputs/inspect the data/set up environments**

Import and add libraries required to complete anaylsis

In [13]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import geopy # library to locate the coordinates of addresses, cities, countries, and landmarks
import folium # library that helps you create several types of Leaflet maps
import matplotlib.pyplot as plt #mapping
import seaborn as sns #stats and data visualisation
%matplotlib inline
import lxml # package to read html from website
import matplotlib.pyplot
##conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter
## !conda install -c conda-forge folium=0.5.0 --yes 
import folium # mapping 
import geopy #library to get coordinates
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from pandas.io.json import json_normalize # read json and pandas

Set up my **Foursquare** account

In [3]:
CLIENT_ID = 'Z01G5UOR0Q1WVBHYJXF0AX2FLJENVLGZ2WQSV0VDPDJ0SLJR' # your Foursquare ID
CLIENT_SECRET = 'EMIJH3XPNGDDDV2ZZEUT5G2VYFFYTA4NOJJPX05J1X5CYPNQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z01G5UOR0Q1WVBHYJXF0AX2FLJENVLGZ2WQSV0VDPDJ0SLJR
CLIENT_SECRET:EMIJH3XPNGDDDV2ZZEUT5G2VYFFYTA4NOJJPX05J1X5CYPNQ


Retrive the COVID-19 case history from the **Dept of Health & Human Services** website daily briefing page

In [7]:
## Get the URL and read the dataframe
url = 'https://www.dhhs.vic.gov.au/coronavirus-update-victoria-05-june-2020'
vic_url = requests.get(url)
vic_url

<Response [200]>

Read the data from the website

In [8]:
vic_data = pd.read_html(vic_url.text)
vic_data

[                                      0                                    1  \
 0   Confirmed cases by LGA, 5 June 2020  Confirmed cases by LGA, 5 June 2020   
 1                                   LGA         Total confirmed cases (ever)   
 2                             MELBOURNE                                  108   
 3                           STONNINGTON                                   94   
 4                               BANYULE                                   91   
 ..                                  ...                                  ...   
 72                         WEST WIMMERA                                    1   
 73                              WODONGA                                    1   
 74                         YARRIAMBIACK                                    1   
 75                              UNKNOWN                                    5   
 76                                TOTAL                                 1681   
 
                          

In [51]:
v_d = pd.read_html('https://www.dhhs.vic.gov.au/coronavirus-update-victoria-07-june-2020')
type(v_d)
len(v_d)
df=v_d[0]
df

,0,1,2
0,Confirmed cases by LGA 7 June 2020,NaN,NaN
1,LGA,Total confirmed cases (ever),Currently active cases
2,MELBOURNE,109,11
3,STONNINGTON,94,0
4,BANYULE,91,1
...,...,...,...
72,WEST WIMMERA,1,0
73,WODONGA,1,0
74,YARRIAMBIACK,1,0
75,UNKNOWN,4,4


**2. Explore and understand the data**

Create a Dataframe, name the columns and interpret the data types available

In [52]:
df.head(5)
df.columns = ['LGA', 'Total confirmed cases (ever)', 'Confirmed cases by LGA, 7 June 2020']
df.head()
df.drop(df.index[[0,1,]], inplace=True)
df.head()
df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 2 to 76
Data columns (total 3 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   LGA                                  75 non-null     object
 1   Total confirmed cases (ever)         75 non-null     object
 2   Confirmed cases by LGA, 7 June 2020  75 non-null     object
dtypes: object(3)
memory usage: 2.3+ KB


LGA                                    object
Total confirmed cases (ever)           object
Confirmed cases by LGA, 7 June 2020    object
dtype: object

**3. Cleanse and enhance the data**

Convert the data types from "Object" to "Int"

In [53]:
df[["Total confirmed cases (ever)"]] = df[["Total confirmed cases (ever)"]].astype("int")
df[["Confirmed cases by LGA, 7 June 2020"]] = df[["Confirmed cases by LGA, 7 June 2020"]].astype("int")
df.dtypes

LGA                                    object
Total confirmed cases (ever)            int64
Confirmed cases by LGA, 7 June 2020     int64
dtype: object

Confirm the new types 

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 2 to 76
Data columns (total 3 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   LGA                                  75 non-null     object
 1   Total confirmed cases (ever)         75 non-null     int64 
 2   Confirmed cases by LGA, 7 June 2020  75 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ KB


Drop the last row from the table 

In [55]:
df.drop([76], axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 2 to 75
Data columns (total 3 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   LGA                                  74 non-null     object
 1   Total confirmed cases (ever)         74 non-null     int64 
 2   Confirmed cases by LGA, 7 June 2020  74 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ KB


In [56]:
df.head()

,LGA,Total confirmed cases (ever),"Confirmed cases by LGA, 7 June 2020"
2,MELBOURNE,109,11
3,STONNINGTON,94,0
4,BANYULE,91,1
5,BOROONDARA,73,2
6,BRIMBANK,73,7


Create bins to assign the muncipalities to a case_level.<br>
The case_level is automatically applied and added to the dataframe <br>
**High** or <br>
**Medium** or <br>
**Low** <br>
case counts

In [57]:
bins = np.linspace(min(df["Total confirmed cases (ever)"]), max(df["Total confirmed cases (ever)"]),4)
group_names = ["Low", "Medium", "high"]
df["case_level"]=pd.cut(df["Total confirmed cases (ever)"],bins, labels = group_names, include_lowest=True)
df

,LGA,Total confirmed cases (ever),"Confirmed cases by LGA, 7 June 2020",case_level
2,MELBOURNE,109,11,high
3,STONNINGTON,94,0,high
4,BANYULE,91,1,high
5,BOROONDARA,73,2,Medium
6,BRIMBANK,73,7,Medium
...,...,...,...,...
71,MOYNE,1,0,Low
72,WEST WIMMERA,1,0,Low
73,WODONGA,1,0,Low
74,YARRIAMBIACK,1,0,Low


Testing the **Foursquares API** to retrieve Longitude and Latitude for muncipalities with **high** case levels

In [25]:
address = 'Melbourne, VIC'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.8142176 144.9631608


In [37]:
address = 'stonnington, VIC'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.8613851 145.03914073484043


In [38]:
address = 'Bayside, VIC'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.93978075 145.01775256216214


In [39]:
address = 'Kingston, VIC'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.374292 143.9558066


In [40]:
address = 'Brimbank, VIC'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-37.7427263 144.80547291877602


Use the Nominatim to retrieve street addresses

In [45]:
geolocator = Nominatim()

/Users/jewelspace/.conda/envs/Final_Project/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


Use the **Lamda** function to apply the LGA (muncipalities) to add the Latitude and Longitude co-ordinates to the dataframe

In [58]:
df['LGA_coord'] = df['LGA'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [59]:
df

,LGA,Total confirmed cases (ever),"Confirmed cases by LGA, 7 June 2020",case_level,LGA_coord
2,MELBOURNE,109,11,high,"(-37.8142176, 144.9631608)"
3,STONNINGTON,94,0,high,"(-37.8613851, 145.03914073484043)"
4,BANYULE,91,1,high,"(-37.7470917, 145.0812596)"
5,BOROONDARA,73,2,Medium,"(-37.8268263, 145.06120331054495)"
6,BRIMBANK,73,7,Medium,"(-37.7427263, 144.80547291877602)"
...,...,...,...,...,...
71,MOYNE,1,0,Low,"(52.73923215, -7.704720069331158)"
72,WEST WIMMERA,1,0,Low,"(-37.27908815, 142.98062447634112)"
73,WODONGA,1,0,Low,"(-36.115235, 146.9051519243323)"
74,YARRIAMBIACK,1,0,Low,"(-36.0262596, 142.38850464811537)"


Apply the latitude and longitudes to the dataframe

In [61]:
df[['latitude', 'longitude']] = df['LGA_coord'].apply(pd.Series)

In [62]:
df

,LGA,Total confirmed cases (ever),"Confirmed cases by LGA, 7 June 2020",case_level,LGA_coord,latitude,longitude
2,MELBOURNE,109,11,high,"(-37.8142176, 144.9631608)",-37.814218,144.963161
3,STONNINGTON,94,0,high,"(-37.8613851, 145.03914073484043)",-37.861385,145.039141
4,BANYULE,91,1,high,"(-37.7470917, 145.0812596)",-37.747092,145.081260
5,BOROONDARA,73,2,Medium,"(-37.8268263, 145.06120331054495)",-37.826826,145.061203
6,BRIMBANK,73,7,Medium,"(-37.7427263, 144.80547291877602)",-37.742726,144.805473
...,...,...,...,...,...,...,...
71,MOYNE,1,0,Low,"(52.73923215, -7.704720069331158)",52.739232,-7.704720
72,WEST WIMMERA,1,0,Low,"(-37.27908815, 142.98062447634112)",-37.279088,142.980624
73,WODONGA,1,0,Low,"(-36.115235, 146.9051519243323)",-36.115235,146.905152
74,YARRIAMBIACK,1,0,Low,"(-36.0262596, 142.38850464811537)",-36.026260,142.388505


**4. Model the data**

Set the reference point a **Melbourne CBD** from Victoria as this muncipality had the highest number of recorded COVID-19 cases

In [63]:
address = 'Melbourne, Vic'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne, Victoria are {}, {}.'.format(latitude, longitude))

/Users/jewelspace/.conda/envs/Final_Project/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Melbourne, Victoria are -37.8142176, 144.9631608.


Map of cases areas, **clicking on the markers** users can quickly identify areas and their case level

In [68]:
# create map of Melbourne using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, LGA, case_level in zip(df['latitude'], df['longitude'], df['LGA'], df['case_level']):
    label = '{}, {}'.format(LGA, case_level)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

Create a search for **Gym and Fitness Centres**

In [78]:
search_query = 'Gym / Fitness Center'
radius = 1500
print(search_query + ' .... OK!')

Gym / Fitness Center .... OK!


Use my **Foursquares API** to return a **Gym and Fitness Centres** in Victoria

In [79]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Z01G5UOR0Q1WVBHYJXF0AX2FLJENVLGZ2WQSV0VDPDJ0SLJR&client_secret=EMIJH3XPNGDDDV2ZZEUT5G2VYFFYTA4NOJJPX05J1X5CYPNQ&ll=-37.8142176,144.9631608&v=20180604&query=Gym / Fitness Center&radius=1500&limit=30'

Display the results

In [80]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eec0474b1cac0001bf3e6b7'},
 'response': {'venues': [{'id': '51152834e4b0b800aa983a2d',
    'name': 'Fitness Center',
    'location': {'lat': -37.81145524559534,
     'lng': 144.9761321551663,
     'labeledLatLngs': [{'label': 'display',
       'lat': -37.81145524559534,
       'lng': 144.9761321551663}],
     'distance': 1181,
     'cc': 'AU',
     'country': 'Australia',
     'formattedAddress': ['Australia']},
    'categories': [{'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'shortName': 'Gym / Fitness',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592526063',
    'hasPerk': False},
   {'id': '5d145d6cb55cb80023dd36c7',
    'name': 'FMC Melbourne - Fitness & Martial Arts Center',
    'location': {'address': '446 Collins St',
     'lat': -37.814666152243205,
    

Use the **json normalise** assign the **gym and fitness centre** data from **Foursquares API**  to the new dataframe

In [76]:
from pandas.io.json import json_normalize

In [82]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df

/Users/jewelspace/.conda/envs/Final_Project/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet,location.neighborhood
0,51152834e4b0b800aa983a2d,Fitness Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1592526063,False,-37.811455,144.976132,"[{'label': 'display', 'lat': -37.8114552455953...",1181,AU,Australia,[Australia],NaN,NaN,NaN,NaN,NaN,NaN
1,5d145d6cb55cb80023dd36c7,FMC Melbourne - Fitness & Martial Arts Center,"[{'id': '4bf58dd8d48988d101941735', 'name': 'M...",v-1592526063,False,-37.814666,144.962368,"[{'label': 'display', 'lat': -37.8146661522432...",85,AU,Australia,"[446 Collins St, Melbourne VIC 3000, Australia]",446 Collins St,3000,Melbourne,VIC,NaN,NaN
2,5ceaef76f1936e002c896b55,doubletree fitness center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1592526063,False,-37.817762,144.965690,"[{'label': 'display', 'lat': -37.817762, 'lng'...",452,AU,Australia,"[270 Flinders Street, Melbourne VIC 3000, Aust...",270 Flinders Street,3000,Melbourne,VIC,NaN,NaN
3,5e3df8b23bf82d000731d6fe,Jasper Hotel Fitness Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1592526063,False,-37.807479,144.959799,"[{'label': 'display', 'lat': -37.807479, 'lng'...",806,AU,Australia,"[489 Elizabeth Street (Therry), Melbourne VIC ...",489 Elizabeth Street,3000,Melbourne,VIC,Therry,NaN
4,59a5347d4a1cc05dc1abd154,Gym In Melbourne | Fitness Classes | Gym Bootcamp,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1592526063,False,-37.807460,144.966645,"[{'label': 'display', 'lat': -37.8074604522283...",812,AU,Australia,"[Mackenzie Street, Australia]",NaN,NaN,NaN,NaN,Mackenzie Street,NaN
5,4b0d92f1f964a520554b23e3,RACV Club - Fitness Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1592526063,False,-37.815656,144.959660,"[{'label': 'display', 'lat': -37.8156560733584...",347,AU,Australia,"[501 Bourke St (New Chancery Ln), Melbourne VI...",501 Bourke St,3000,Melbourne,VIC,New Chancery Ln,NaN
6,4f9b081be4b0873b66757d86,Marriott Fitness center,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1592526063,False,-37.799371,144.959871,"[{'label': 'display', 'lat': -37.799371, 'lng'...",1677,AU,Australia,"[Residence Inn, Australia]",Residence Inn,NaN,NaN,NaN,NaN,NaN
7,529e36c4498e392add386dfa,City Club At Grand Hyatt,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1592526063,False,-37.815319,144.969555,"[{'label': 'display', 'lat': -37.8153194787920...",575,AU,Australia,"[111 Collins Street, Melbourne VIC 3000, Austr...",111 Collins Street,3000,Melbourne,VIC,NaN,"Melbourne CBD, Melbourne, VIC"
8,526e2366498ee3fd8486084e,Melbourne University Gym and Sport Center,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1592526063,False,-37.802095,144.961875,"[{'label': 'display', 'lat': -37.8020949273538...",1354,AU,Australia,[Australia],NaN,NaN,NaN,NaN,NaN,NaN
9,4c0c0bdb340720a117128993,RMIT City Fitness,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1592526063,False,-37.808370,144.963289,"[{'label': 'display', 'lat': -37.8083702325557...",651,AU,Australia,"[360 Swanston St., Melbourne VIC 3000, Australia]",360 Swanston St.,3000,Melbourne,VIC,NaN,NaN


Filter the data display the names

In [84]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,neighborhood,id
0,Fitness Center,Gym / Fitness Center,-37.811455,144.976132,"[{'label': 'display', 'lat': -37.8114552455953...",1181,AU,Australia,[Australia],NaN,NaN,NaN,NaN,NaN,NaN,51152834e4b0b800aa983a2d
1,FMC Melbourne - Fitness & Martial Arts Center,Martial Arts Dojo,-37.814666,144.962368,"[{'label': 'display', 'lat': -37.8146661522432...",85,AU,Australia,"[446 Collins St, Melbourne VIC 3000, Australia]",446 Collins St,3000,Melbourne,VIC,NaN,NaN,5d145d6cb55cb80023dd36c7
2,doubletree fitness center,Gym / Fitness Center,-37.817762,144.965690,"[{'label': 'display', 'lat': -37.817762, 'lng'...",452,AU,Australia,"[270 Flinders Street, Melbourne VIC 3000, Aust...",270 Flinders Street,3000,Melbourne,VIC,NaN,NaN,5ceaef76f1936e002c896b55
3,Jasper Hotel Fitness Center,Gym / Fitness Center,-37.807479,144.959799,"[{'label': 'display', 'lat': -37.807479, 'lng'...",806,AU,Australia,"[489 Elizabeth Street (Therry), Melbourne VIC ...",489 Elizabeth Street,3000,Melbourne,VIC,Therry,NaN,5e3df8b23bf82d000731d6fe
4,Gym In Melbourne | Fitness Classes | Gym Bootcamp,Gym / Fitness Center,-37.807460,144.966645,"[{'label': 'display', 'lat': -37.8074604522283...",812,AU,Australia,"[Mackenzie Street, Australia]",NaN,NaN,NaN,NaN,Mackenzie Street,NaN,59a5347d4a1cc05dc1abd154
5,RACV Club - Fitness Centre,Gym,-37.815656,144.959660,"[{'label': 'display', 'lat': -37.8156560733584...",347,AU,Australia,"[501 Bourke St (New Chancery Ln), Melbourne VI...",501 Bourke St,3000,Melbourne,VIC,New Chancery Ln,NaN,4b0d92f1f964a520554b23e3
6,Marriott Fitness center,Gym,-37.799371,144.959871,"[{'label': 'display', 'lat': -37.799371, 'lng'...",1677,AU,Australia,"[Residence Inn, Australia]",Residence Inn,NaN,NaN,NaN,NaN,NaN,4f9b081be4b0873b66757d86
7,City Club At Grand Hyatt,Gym,-37.815319,144.969555,"[{'label': 'display', 'lat': -37.8153194787920...",575,AU,Australia,"[111 Collins Street, Melbourne VIC 3000, Austr...",111 Collins Street,3000,Melbourne,VIC,NaN,"Melbourne CBD, Melbourne, VIC",529e36c4498e392add386dfa
8,Melbourne University Gym and Sport Center,College Gym,-37.802095,144.961875,"[{'label': 'display', 'lat': -37.8020949273538...",1354,AU,Australia,[Australia],NaN,NaN,NaN,NaN,NaN,NaN,526e2366498ee3fd8486084e
9,RMIT City Fitness,College Gym,-37.808370,144.963289,"[{'label': 'display', 'lat': -37.8083702325557...",651,AU,Australia,"[360 Swanston St., Melbourne VIC 3000, Australia]",360 Swanston St.,3000,Melbourne,VIC,NaN,NaN,4c0c0bdb340720a117128993


In [113]:
dataframe_filtered.name

0                                       Fitness Center
1        FMC Melbourne - Fitness & Martial Arts Center
2                            doubletree fitness center
3                          Jasper Hotel Fitness Center
4    Gym In Melbourne | Fitness Classes | Gym Bootcamp
5                           RACV Club - Fitness Centre
6                              Marriott Fitness center
7                             City Club At Grand Hyatt
8            Melbourne University Gym and Sport Center
9                                    RMIT City Fitness
Name: name, dtype: object

Add a map of the gyms in **Melbourne CBD** the area with the **highest COVID-19 cases** in Victoria

In [94]:
gym_map = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around the Melbourne CBD

# add a red circle marker to represent Melbourne CBD
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Melbourne CBD',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(gym_map)

# add the gyms as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(gym_map)

# display map
gym_map

Let's expore a specific gym and see what information we can discover

<h4>Doubletree fitness center<h4>

In [134]:
gym_id = '5d145d6cb55cb80023dd36c7' # Doubletree ID
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(gym_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5d145d6cb55cb80023dd36c7?client_id=Z01G5UOR0Q1WVBHYJXF0AX2FLJENVLGZ2WQSV0VDPDJ0SLJR&client_secret=EMIJH3XPNGDDDV2ZZEUT5G2VYFFYTA4NOJJPX05J1X5CYPNQ&v=20180604'

In [135]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


{'id': '5d145d6cb55cb80023dd36c7',
 'name': 'FMC Melbourne - Fitness & Martial Arts Center',
 'contact': {'phone': '+61390159502', 'formattedPhone': '+61 3 9015 9502'},
 'location': {'address': '446 Collins St',
  'lat': -37.814666152243205,
  'lng': 144.9623680114746,
  'labeledLatLngs': [{'label': 'display',
    'lat': -37.814666152243205,
    'lng': 144.9623680114746}],
  'postalCode': '3000',
  'cc': 'AU',
  'city': 'Melbourne',
  'state': 'VIC',
  'country': 'Australia',
  'formattedAddress': ['446 Collins St', 'Melbourne VIC 3000', 'Australia']},
 'canonicalUrl': 'https://foursquare.com/v/fmc-melbourne--fitness--martial-arts-center/5d145d6cb55cb80023dd36c7',
 'categories': [{'id': '4bf58dd8d48988d101941735',
   'name': 'Martial Arts Dojo',
   'pluralName': 'Martial Arts Dojos',
   'shortName': 'Martial Arts',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_martialarts_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount'

In [137]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Let's explore for a gym in another location

In [4]:
gym_id_1 = '51152834e4b0b800aa983a2d' # Grand Hyatt Gym
url_1 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(gym_id_1, CLIENT_ID, CLIENT_SECRET, VERSION)
url_1

'https://api.foursquare.com/v2/venues/51152834e4b0b800aa983a2d?client_id=Z01G5UOR0Q1WVBHYJXF0AX2FLJENVLGZ2WQSV0VDPDJ0SLJR&client_secret=EMIJH3XPNGDDDV2ZZEUT5G2VYFFYTA4NOJJPX05J1X5CYPNQ&v=20180604'

In [14]:
result_1 = requests.get(url_1).json()
print(result_1['response']['venue'].keys())
result_1['response']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'venue': {'id': '51152834e4b0b800aa983a2d',
  'name': 'Fitness Center',
  'contact': {},
  'location': {'lat': -37.81145524559534,
   'lng': 144.9761321551663,
   'labeledLatLngs': [{'label': 'display',
     'lat': -37.81145524559534,
     'lng': 144.9761321551663}],
   'cc': 'AU',
   'country': 'Australia',
   'formattedAddress': ['Australia']},
  'canonicalUrl': 'https://foursquare.com/v/fitness-center/51152834e4b0b800aa983a2d',
  'categories': [{'id': '4bf58dd8d48988d175941735',
    'name': 'Gym / Fitness Center',
    'pluralName': 'Gyms or Fitness Centers',
    'shortName': 'Gym / Fitness',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 3},
  'likes': {'count': 3,
   'groups': [{'type': 'others',
     'count': 3,
     'items': [{'id': '40678559',
       'firstName': 'A.Alper',
       'lastName': 'G',
       'photo': {'prefix': 'https://fastly.4sqi.net/img/use

In [15]:
try:
    print(result_1['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


What a shame it appears this venue hasnt been rated either

I will bring in some additional data from the *Aus gov website*

In [22]:
path='http://data.gov.au/storage/f/2013-05-12T181635/tmp6iSsn4fitness-centres.csv'
df_2 = pd.read_csv(path)
df_2.head()

,Name,Address,Suburb,Postcode,State,Business Category,LGA,Region
0,Action Fitness Club,63 Mortlake Road,Warrnambool,3280,VIC,Fitness centre,Warrnambool (C),Barwon South Western Region
1,Active Lifestyle Specialists,374A Warrugal Road,Ashburton,3147,VIC,Fitness centre,Monash (C); Boroondara (C),Eastern Metropolitan Region
2,Alexandra Community Leisure Centre,Community Centre,Alexandra,3714,VIC,Fitness centre,Murrindindi (S),Hume Region
3,Ballarat YMCA,122 Armstrong Street,Ballarat South,3350,VIC,Fitness centre,Ballarat (C),Grampians Region
4,Bass Coast YMCA,c/- 10 -12 Church Street,Cowes,3922,VIC,Fitness centre,Bass Coast,Gippsland


In [25]:
geolocator = Nominatim()

NameError: name 'Nominatim' is not defined

In [23]:
df_2['LGA'] = df_2['LGA'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

NameError: name 'geolocator' is not defined

In [128]:
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline